In [246]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from api_key_file import api_key
from IPython.display import JSON
import requests
import json
import pandas as pd
import csv

api_service_name = "youtube"
api_version = "v3"

In [35]:
channel_ids = ["UCLKZ20yD2tNMBOkSDZo4FeQ",
"UCzAF54cHk1ZO82af-8E3qOQ",
"UCteRPiisgIoHtMgqHegpWAQ",
"UCw_LFe2pS8x3NyipGNJgeEA",
"UCLLw7jmFsvfIVaUFsLs8mlQ",
"UCDybamfye5An6p-j1t2YMsg",
"UC7cs8q-gJRlGwj4A8OmCmXg",
"UCQiNyL7ik4FIlV2UCvojq0g",
"UCJQJAI7IjbLcpsjWdSzYz0Q",
"UCu1xbgCV5o48h_BYCQD7KJg",
"UCggXQcy57bwxUeZI1LB8DCw",
"UCPcyffOFc7YGyDgqmvZJs7Q",
"UCC8nkE-p-7S0ZPJXrEs2QNA",
"UCcJQ96WlEhJ0Ve0SLmU310Q",
"UCXqKK0ZZO5EhS6USGkGcuiw",
"UCLpovxJVLBZrXJGCymB6LYw",
"UC2fYCeDDpXrQVjSPdJY4lyA",
"UC4MZ7zUHb5eAxU75Dc_nqdQ",
"UC8butISFwT-Wl7EV0hUK0BQ",
"UCsvqVGtbbyHaMoevxPAq9Fg",
"UCilJYqsETFZ7NG9g1xVKfcQ",
"UCsLo154Krjwhoz8W00N8ItA",
"UCsd0xmaXyUGpD35IX4xrYEw",
"UCv3N90AXQxUtaSnKSqOoOpA",
"UCmTS2gGo4gjZLt8LxVnF3XA",
"UCTHAIlSOqquwOKnd_nCJgAQ",
"UCWI-ohtRu8eEeDj93hmUsUQ",
"UC50JofjMZpr3KCbOx6dHi_A",
"UCUOMtzQmEmhIAF8G_cha_Xw",
"UCdngmbVKX1Tgre699-XLlUA",
"UC5qEPWtHLFRIjhW_3xd5g2A",
"UC2UXDak6o7rBm23k3Vv5dww",
"UCyUBW72KU30dfAYWLVNZO8Q",
"UCVe8CMJF4caRzuckVYV8CaQ",
"UCvDqylEzCZJHNea6k1XMkQw",
"UCEDkO7wshcDZ7UZo17rPkzQ",
"UCt-Pwe2fODjH4Wuwf5VqE7A",
"UClLXKYEEM8OBBx85DOa6-cg",
"UCN0u98NPeC0AW3pD6MbVeNg",
"UCwAnu01qlnVg1Ai2AbtTMaA",
"UCthRsjtSd2ZTrfgjgnBXMPw",
"UCx3ZYdyVr8BUQDIhS1QggxA",
"UCo2avEF03abQmCXdU_eUGkg",
"UC4xKdmAXFh4ACyhpiQ_3qBw",
"UCIrOHHdl6C4IWv96N0EaHMA",
"UCJNUtbsYnt-8meFSj_yF50Q",
"UCEHFikgnRuLd1HYKTLrae9Q",
"UC1nBZreWJArdFgiIu3BTeGw",
"UC3kFmMxz3IdLdz4TxA9a4DQ",
"UCpa0VHHeSUV1D17X8MoX5xA"]

In [36]:
len(channel_ids)

50

In [37]:
# Get credentials and create an API client
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

In [38]:
# channel request (getting all upload playlist ids from 50 selected tech career focused channels)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_ids)
)
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [47]:
# get upload playlists

upload_ids = [item["contentDetails"]["relatedPlaylists"]["uploads"] for item in response["items"]]

In [99]:
upload_ids_test = ["UU3kFmMxz3IdLdz4TxA9a4DQ", "UUilJYqsETFZ7NG9g1xVKfcQ", "UUN0u98NPeC0AW3pD6MbVeNg"]

In [100]:
video_ids = []
temp = []

In [244]:
for upload_playlist_id in upload_ids:
    
    #initial request
    request = youtube.playlistItems().list(
    part=['snippet'], 
    maxResults=50, 
    playlistId=upload_playlist_id)  #id=','.join(upload_ids))
    response = request.execute()

    #try to get all of the video id's from the response
    try:
        temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        video_ids = video_ids + temp
    except Exception as e:
        print(e)

    #increment page token
    next_page_token = response.get('nextPageToken')
    #print(next_page_token)
    #next_page_token
    
    # https://stackoverflow.com/questions/30653208/retrieve-video-ids-contained-in-a-playlist-youtube-api-v3
    # while there are still pages in the response to parse
    while next_page_token is not None: #and len(video_ids) < 100:
        #print(next_page_token)
        #make the same request, just get the next page of uploads (using nextPageToken which stores the token of the next page of search results
        request = youtube.playlistItems().list(
        part=['snippet'], 
        maxResults=50, 
        playlistId=upload_playlist_id,
        #id=','.join(upload_ids_test),
        pageToken = next_page_token)#id=','.join(upload_ids))

        response = request.execute()
        #make a temporary list of video ids on this page
        try:
            temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        except Exception as e:
            print(e)

        # add to video id list
        video_ids = video_ids + temp

        #increment page token
        next_page_token = response.get('nextPageToken')
        #print(next_page_token)

In [245]:
len(video_ids)

27499

In [ ]:
def concatenate_df(response_json, output_df):
    for video in response_json:


        row_to_add = {"Video ID":video["id"], "Channel Title": video["snippet"]["channelTitle"], "Tags": video["snippet"]["tags"],\
                      "Tag Count": len(video["snippet"]["tags"]), "Published At": video["snippet"]["publishedAt"], "Channel ID": video["snippet"]["channelId"], \
                      "Title": video["snippet"]["title"],"Category ID": video["snippet"]["categoryId"], "Description Length (words)": len(video["snippet"]["description"].split(" ")), \
                      "Description Length (characters)": len(video["snippet"]["description"]), "Duration": video["contentDetails"]["duration"], "Dimension": video["contentDetails"]["dimension"], \
                     "Definition": video["contentDetails"]["definition"], "Caption": video["contentDetails"]["caption"], "Licensed Content (T/F)": video["contentDetails"]["licensedContent"], \
                     "Projection": video["contentDetails"]["projection"], "View Count": video["statistics"]["viewCount"], "Like Count": video["statistics"]["likeCount"], \
                     "Favorite Count": video["statistics"]["favoriteCount"], "Comment Count": video["statistics"]["commentCount"]}

        output_df = pd.concat( [ output_df, pd.DataFrame.from_records([row_to_add]) ], ignore_index=True )
    return output_df

In [247]:
#https://blog.enterprisedna.co/how-to-write-a-list-to-csv-in-python/
#save video id's to file just in case
with open('video_ids.csv', 'w', newline='') as file:
    # Step 4: Using csv.writer to write the list to the CSV file
    writer = csv.writer(file)
    writer.writerow(video_ids) # Use writerow for single list

In [237]:
#video statistics request

# request = youtube.videos().list(
#     part="snippet,contentDetails,statistics",
#     id=','.join(video_ids[0:len(video_ids)-619])
# )


for i in range(0, 15, 50):
    
    #to avoid list index out of bounds error (dataset not divisible by 50)
    increment = i+50 if (i+50 < len(video_ids)-1) else len(video_ids)-1
    
    request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id=','.join(video_ids[i, increment]),
    maxResults=50,
    )
    
    response = request.execute()


    final_df = concatenate_df(response["items"], final_df)

    

<IPython.core.display.JSON object>

In [248]:
def concatenate_df(response_json, output_df):
    for video in response_json:


        row_to_add = {"Video ID":video["id"], "Channel Title": video["snippet"]["channelTitle"], "Tags": video["snippet"]["tags"],\
                      "Tag Count": len(video["snippet"]["tags"]), "Published At": video["snippet"]["publishedAt"], "Channel ID": video["snippet"]["channelId"], \
                      "Title": video["snippet"]["title"],"Category ID": video["snippet"]["categoryId"], "Description Length (words)": len(video["snippet"]["description"].split(" ")), \
                      "Description Length (characters)": len(video["snippet"]["description"]), "Duration": video["contentDetails"]["duration"], "Dimension": video["contentDetails"]["dimension"], \
                     "Definition": video["contentDetails"]["definition"], "Caption": video["contentDetails"]["caption"], "Licensed Content (T/F)": video["contentDetails"]["licensedContent"], \
                     "Projection": video["contentDetails"]["projection"], "View Count": video["statistics"]["viewCount"], "Like Count": video["statistics"]["likeCount"], \
                     "Favorite Count": video["statistics"]["favoriteCount"], "Comment Count": video["statistics"]["commentCount"]}

        output_df = pd.concat( [ output_df, pd.DataFrame.from_records([row_to_add]) ], ignore_index=True )
    return output_df

In [220]:
final_df

,Video ID,Channel Title,Tags,Tag Count,Published At,Channel ID,Title,Category ID,Description Length (words),Description Length (characters),Duration,Dimension,Definition,Caption,Licensed Content (T/F),Projection,View Count,Like Count,Favorite Count,Comment Count
0,OfAcLjgQUcQ,PushToProd,"[Software, Networking, NOC, network analyst, C...",36,2019-08-03T05:07:07Z,UCN0u98NPeC0AW3pD6MbVeNg,Day In the Life of an IT System Admin (Security),22,89,451,PT13M4S,2d,hd,false,True,rectangular,84849,1939,0,230


In [41]:
# # videos by keyword request
# temp = []
# # Get credentials and create an API client

# request = youtube.search().list(
#     part="snippet",
#     maxResults=50,
#     q="tech bootcamp"
# )
# response = request.execute()

# #get first 50 video id's and create list
# try:
#     video_ids = [video["id"]["videoId"] for video in response["items"]]
# except Exception as e:
#     print(e)

# #increment page token
# next_page_token = response.get('nextPageToken')

# # while there are still pages in the response to parse
# while next_page_token is not None and len(video_ids) < 100:
#     request = youtube.search().list(
#     part="snippet",
#     maxResults=51,
#     q="tech bootcamp",
#     pageToken = next_page_token # get results from the page after previous request
# )
    
#     #make a temporary list of video ids on this page
#     try:
#         temp = [video["id"]["videoId"] for video in response["items"]]
#     except Exception as e:
#         print(e)
    
#     # add to video id list
#     video_ids = video_ids + temp
    
#     #increment page token
#     next_page_token = response.get('nextPageToken')

# JSON(response)

# video_ids

In [81]:
# try:
#     video_ids = [video["id"]["videoId"] for video in response["items"]]
# except Exception as e:
#     print(e)

In [43]:
# #video statistics request

# request = youtube.videos().list(
#     part="snippet,contentDetails,statistics",
#     id=','.join(video_ids)
# )
# response = request.execute()

# JSON(response)

In [44]:
# likes = [video["statistics"]["likeCount"] for video in response["items"]]
# likes

In [45]:
# #channel id by name
# request = youtube.channels().list(
#     part="snippet,contentDetails,statistics",
#     forUsername="@JeffSu"
# )
# response = request.execute()

# JSON(response)

In [ ]:
# request = youtube.channels().list(
#     part="snippet,contentDetails,statistics",
#     id="UC_x5XG1OV2P6uZZ5FSM9Ttw"
# )
# response = request.execute()

# print(response)